# Display a layout of the Gaussian Elimination Algorithm

In [1]:
using PrettyTables
#using LaTeXStrings
# --------------------------------------------------------------------------------------------------
"""
pt_frac(value)

HTML format of Julia Fractions

# Example:
```julia-repl
julia> pt_frac(-3//5)
3 / 5
```
"""
function pt_frac(value)
    n,d=numerator(value),denominator(value)
    d == 1 ? "$n" : "$n &frasl; $d"
end
# --------------------------------------------------------------------------------------------------
"""
ge_layout( A, layers )

Display a computational layout for a Gaussian Elimination Computation

# Arguments
- `n::Array`:         an array of integers or rational numbers
- `layers::Array`:    an array of the computations
- `to_str::Function`: a function converting a matrix entry to a string

# Example:
```julia-repl
julia> ge_layout( A, [ E1 A1; E2 A2; E3 A3], to_str = pt_real") # E_{i} is the i'th elemination matrix, A_i = E_i * A_{i-1}
the layout
```
"""
function ge_layout( A, layers, to_str = pt_frac )
    n  = size(A, 1)
    tf = HTMLTableFormat(css=""".verticalline { border-right: 1px solid black; height: 100%; }""" )
    #sp = fill("&nbsp;", n+size(layers,1),1 )
    sp = fill("<span class=verticalline></span>", n+size(layers,1), 1)
    MA = vcat( hcat( fill("", n,n), map(to_str, A)),
                     map( to_str,  layers))
    ndx = [2:(n+1); 1; (n+2):(size(layers,2)+1)]
    M  = [sp MA][:, ndx]

    bg_gray_1    = HTMLHighlighter((data,i,j)->iseven((i-1)÷n+1)        && j>n,  HTMLDecoration(background = "#F0F0F0", color = "blue"  ))
    bg_gray_2    = HTMLHighlighter((data,i,j)->isodd( (i-1)÷n+1)        && j>n,  HTMLDecoration(background = "#FBFAFA", color = "blue"  ))

    bg_yellow_1  = HTMLHighlighter((data,i,j)->iseven((i-1)÷n+1)        && j<=n, HTMLDecoration(background = "#FDFAD4", color = "blue"  ))
    bg_yellow_2  = HTMLHighlighter((data,i,j)->i>n && isodd( (i-1)÷n+1) && j<=n, HTMLDecoration(background = "#F6EFA1", color = "blue"  ))
    bg_white     = HTMLHighlighter((data,i,j)->i<=n                     && j<=n, HTMLDecoration(background = "#FFFFFF", color = "blue"  ))
    pretty_table( M, noheader = true, highlighters=(bg_white,bg_gray_1,bg_gray_2,bg_yellow_1,bg_yellow_2), tf=tf, backend = :html )
end

ge_layout

In [2]:
A  = [ 1 3 1 4;
      -2 5 5 6;
       1 3 1 5;
       2 2 8 2
     ]
E1 = [ 1 0 0 0;
       2 1 0 0;
      -1 0 1 0;
      -2 0 0 1
    ]
A1 = E1 * A

E2 = [ 1 0 0 0;
       0 1 0 0;
       0 0 1 0;
       0 4//11 0 1
    ]
A2 = E2 * A1

E3 = [ 1 0 0 0;
       0 1 0 0;
       0 0 0 1;
       0 0 1 0
    ]
A3 = E3 * A2

ge_layout( A, [E1 A1; E2 A2; E3 A3])

,,,,,1,3,1,4
,,,,,-2,5,5,6
,,,,,1,3,1,5
,,,,,2,2,8,2
1,0,0,0,,1,3,1,4
2,1,0,0,,0,11,7,14
-1,0,1,0,,0,0,0,1
-2,0,0,1,,0,-4,6,-6
1,0,0,0,,1,3,1,4
0,1,0,0,,0,11,7,14
0,0,1,0,,0,0,0,1


In [3]:
?ge_layout

search: ge_layout



ge_layout( A, layers )

Display a computational layout for a Gaussian Elimination Computation

# Arguments

  * `n::Array`:         an array of integers or rational numbers
  * `layers::Array`:    an array of the computations
  * `to_str::Function`: a function converting a matrix entry to a string

# Example:

```julia-repl
julia> ge_layout( A, [ E1 A1; E2 A2; E3 A3], to_str = pt_real") # E_{i} is the i'th elemination matrix, A_i = E_i * A_{i-1}
the layout
```


In [4]:
ge_layout(A, [E1 A1; E2 A2; E3 A3], value->"$(round(Float64(value), digits=3))")

,,,,,1.0,3.0,1.0,4.0
,,,,,-2.0,5.0,5.0,6.0
,,,,,1.0,3.0,1.0,5.0
,,,,,2.0,2.0,8.0,2.0
1.0,0.0,0.0,0.0,,1.0,3.0,1.0,4.0
2.0,1.0,0.0,0.0,,0.0,11.0,7.0,14.0
-1.0,0.0,1.0,0.0,,0.0,0.0,0.0,1.0
-2.0,0.0,0.0,1.0,,0.0,-4.0,6.0,-6.0
1.0,0.0,0.0,0.0,,1.0,3.0,1.0,4.0
0.0,1.0,0.0,0.0,,0.0,11.0,7.0,14.0
0.0,0.0,1.0,0.0,,0.0,0.0,0.0,1.0
